#<font color=blue>**YOLO를 통한 버클리 데이터셋 학습**</font>

##<font color=green>**구글 드라이브 연결**</font>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##<font color=green>**구글 드라이브 경로로 이동**</font>

In [2]:
cd drive/MyDrive/

/content/drive/MyDrive


##<font color=green>**git의 Darknet_YOLO 설치**</font>

In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14665, done.
remote: Total 14665 (delta 0), reused 0 (delta 0), pack-reused 14665
Receiving objects: 100% (14665/14665), 13.24 MiB | 8.77 MiB/s, done.
Resolving deltas: 100% (9982/9982), done.
Checking out files: 100% (2027/2027), done.


##<font color=green>**OPENCV, GPU 사용 여부 설정**</font>

In [4]:
%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/drive/My Drive/darknet


##<font color=green>**YOLO 빌드**</font>

In [5]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

##<font color=green>**버클리 데이터 업로드 진행**</font>

In [7]:
%cd data/
!gdown --id 18kqQF34wprRZUjSTz17kZkoZN4SRLV0D --output BDD_data.zip

/content/drive/My Drive/darknet/data
Downloading...
From: https://drive.google.com/uc?id=1_MLsDHKmoPSLBe3WuN1jrLWAvQoaOvQ2
To: /content/drive/My Drive/darknet/data/BDD_data.zip
346MB [00:04, 81.5MB/s]


####<font color=red>업로드한 BDD_data.zip 압축풀기

In [8]:
!unzip BDD_data.zip
%cd ..

Streaming output truncated to the last 5000 lines.
  inflating: train_data/1c771b09-095562f4.txt  
  inflating: train_data/1c77f7ba-8372850d.jpg  
  inflating: train_data/1c77f7ba-8372850d.txt  
  inflating: train_data/1c78dab4-1c81edd1.jpg  
  inflating: train_data/1c78dab4-1c81edd1.txt  
  inflating: train_data/1c78dab4-fef00ae7.jpg  
  inflating: train_data/1c78dab4-fef00ae7.txt  
  inflating: train_data/1c79d3e2-2dddf749.jpg  
  inflating: train_data/1c79d3e2-2dddf749.txt  
  inflating: train_data/1c79d3e2-2e4f9b69.jpg  
  inflating: train_data/1c79d3e2-2e4f9b69.txt  
  inflating: train_data/1c7b07e5-32fdee4c.jpg  
  inflating: train_data/1c7b07e5-32fdee4c.txt  
  inflating: train_data/1c7b07e5-7f51545b.jpg  
  inflating: train_data/1c7b07e5-7f51545b.txt  
  inflating: train_data/1c7b07e5-83d597f7.jpg  
  inflating: train_data/1c7b07e5-83d597f7.txt  
  inflating: train_data/1c7b07e5-a7375be5.jpg  
  inflating: train_data/1c7b07e5-a7375be5.txt  
  inflating: train_data/1c7b07e5-de85

UsageError: Line magic function `%cd..` not found.


##<font color=green>**YOLO 학습 파라미터 수정**</font>
drive/MyDrive/darknet/data 폴더안에 있는 yolov3-bdd100k.cfg 파일의 파라미터 값을 수정

/content/drive/MyDrive/darknet/data 폴더 안의 yolov3-bdd100k.cfg 파일의 파라미터를 수정

파라미터 값은 PDF 강의 자료 참고

##<font color=green>**버클리 데이터 YOLO 학습 진행**
Loss 그래프는 실시간으로 확인 X

In [10]:
!./darknet detector train data/bdd.data data/yolov3-bdd100k.cfg data/darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3-bdd100k
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    352 x 352 x   3 ->  352 x 352 x  32 0.214 BF
   1 conv     64       3 x 3/ 2    352 x 352 x  32 ->  176 x 176 x  64 1.142 BF
   2 conv     32       1 x 1/ 1    176 x 176 x  64 ->  176 x 176 x  32 0.127 BF
   3 conv     64       3 x 3/ 1    176 x 176 x  32 ->  176 x 176 x  64 1.142 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 176 x 176 x  64 0.002 BF
   5 conv    128       3 x 3/ 2    176 x 176 x  64 ->   88 x  88 x 128 1.142 BF
   6 conv     64       1 x 1/ 1     88 x  88 x 128 ->   88 x  88 x  64 0.127 BF
   7 conv    128       3 x 3/ 1     88 x  88 x  64 ->   88 x  88 x 128 1.142 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs:  88 x  

####<font color=red>학습된 Loss 그래프 확인

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

imShow('chart.png')

####<font color=red>대회 결과 다운로드</font>
Loss 그래프

파라미터 설정한 yolov3-bdd100k.cfg

학습되어 도출된 weight값

In [ ]:
from google.colab import files
files.download('chart.png')
files.download('data/yolov3-bdd100k.cfg')
files.download('backup/yolov3-bdd100k_final.weights')